How weather is impacting the electricity demand in Victoria?
1.	Check if there is a correlation between Max temperature and Electricity demand for CITIPOWER distributor.
H1 : When Max temperature is > T_Median electricity demand is > D_Median
H0 : No such relationship

2.	Do the similar analysis for Rain and Solar

In [111]:
import pandas as pd
import dask.dataframe as dd
import datetime
import glob
import calendar

In [112]:
def refactor_weather_data(bomDF,year):    
    dateslist = pd.date_range(start=f'1/1/{year}', end=f'31/12/{year}')
    weatherlist  = bomDF['Jan'].tolist()
    if calendar.isleap(year):
        weatherlist = weatherlist + bomDF['Feb'].tolist()[0:29]
    else:
        weatherlist = weatherlist + bomDF['Feb'].tolist()[0:28]          
    weatherlist = weatherlist + bomDF['Mar'].tolist() + bomDF['Apr'].tolist()[0:30] + bomDF['May'].tolist() + bomDF['Jun'].tolist()[0:30] + bomDF['Jul'].tolist() + bomDF['Aug'].tolist() + bomDF['Sep'].tolist()[0:30] + bomDF['Oct'].tolist() + bomDF['Nov'].tolist()[0:30] + bomDF['Dec'].tolist()

    weatherDF = pd.DataFrame({'Date': dateslist, 'Measure': weatherlist})

    return weatherDF


In [113]:

dir = 'BOM/Resources/Weather/_*.csv'
files = glob.glob(dir)
weatherDF = pd.DataFrame()
for f in files:
    filenamelist = f.split("_")
    df = refactor_weather_data(pd.read_csv(f),int(filenamelist[2]))
    df['Distributor'] = filenamelist[1] 
    if filenamelist[3] == '122':
        df['MeasurementType'] = 'MaxTemp'
    elif filenamelist[3] == '123':
        df['MeasurementType'] = 'MinTemp'
    elif filenamelist[3] == '136':
        df['MeasurementType'] = 'Rain Fall'
    elif filenamelist[3] == '193':
        df['MeasurementType'] = 'Solar Exposure'          
    weatherDF = weatherDF.append(df,ignore_index=True)
weatherDF.to_csv('BOM/Resources/Weather/weather.csv')

In [114]:
weatherDF.head()

,Date,Measure,Distributor,MeasurementType
0,2015-01-01,26.2,AusNet,MaxTemp
1,2015-01-02,32.5,AusNet,MaxTemp
2,2015-01-03,38.9,AusNet,MaxTemp
3,2015-01-04,25.4,AusNet,MaxTemp
4,2015-01-05,27.3,AusNet,MaxTemp


In [115]:
# Load all demand data from 2014 to 2019
demand_df = dd.read_csv('Meter data\VIC_20*.csv')
demand_df.head()

,SETTD,PROFILEAREA,DAILYT,VAL01,VAL02,VAL03,VAL04,VAL05,VAL06,VAL07,...,VAL40,VAL41,VAL42,VAL43,VAL44,VAL45,VAL46,VAL47,VAL48,DCTC
0,01/04/2014,CITIPOWER,6363749.701,83948.634,75686.720,70851.453,67607.385,66124.942,64953.942,64306.671,...,160069.821,152968.056,145198.285,133952.389,121154.389,113845.677,111219.265,111446.631,106211.512,MRIM
1,02/04/2014,CITIPOWER,5630825.535,93503.589,84693.357,79590.499,76384.550,74882.906,73439.136,72621.772,...,124538.811,119661.538,113667.967,105081.148,95684.746,91694.112,91698.657,93790.379,89377.501,MRIM
2,03/04/2014,CITIPOWER,5173891.385,77688.455,69851.785,65002.304,62203.224,60863.263,59985.433,59409.226,...,122934.586,118084.353,112122.808,104010.749,94635.246,90432.761,90406.507,92838.823,89320.024,MRIM
3,04/04/2014,CITIPOWER,5044050.180,77761.113,69530.046,64437.280,61304.327,59653.842,58588.591,58218.212,...,114460.060,110398.858,105854.672,100063.838,93741.476,91659.930,93286.271,96403.437,92482.282,MRIM
4,05/04/2014,CITIPOWER,4383318.300,80930.298,72390.242,66461.934,63121.603,60993.088,59567.853,58667.439,...,107937.349,104206.712,100866.709,96744.697,91474.783,91223.017,94194.091,98842.620,95486.413,MRIM
